In [15]:
!pip install torchinfo

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
from torchsummary import summary

import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer

import pandas as pd
import string
import re
from collections import Counter
import nltk

In [42]:
def clean_text(text):
    tokens = text.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    tokens = [re_punc.sub('', w) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if w.lower() not in stop_words]
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

def build_vocab_from_df(df, text_column):
    vocab = Counter()
    for text in df[text_column]:
        tokens = clean_text(text)
        vocab.update(tokens)
    return vocab

In [10]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.lin1 = nn.Linear(10, 10)
        self.lin2 = nn.Linear(10, 20)
        self.lin3 = nn.Linear(20, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, input):
        h1 = F.relu(self.lin1(input))
        h2 = F.relu(self.lin2(h1))
        h3 = F.relu(self.lin3(h2))
        output = torch.sigmoid(self.output(h3))
        return output

model = MLP()

summary(model, input_size=(10,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                   [-1, 20]             220
            Linear-3                   [-1, 10]             210
            Linear-4                    [-1, 1]              11
Total params: 551
Trainable params: 551
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.c2p1 = nn.Conv2d(1, 32, kernel_size=4)
        self.p2c1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c2p2 = nn.Conv2d(32, 16, kernel_size=4)
        self.p2lin = nn.MaxPool2d(kernel_size=2, stride=2)
        self.lin1 = nn.Linear(16 * 13 * 13, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, input):
        c1 = F.relu(self.c2p1(input))
        p1 = self.p2c1(c1)
        c2 = F.relu(self.c2p2(p1))
        p2 = self.p2lin(c2)
        p2_flat = p2.view(-1, 16 * 13 * 13)
        h1 = F.relu(self.lin1(p2_flat))
        output = torch.sigmoid(self.output(h1))
        return output

model = CNN()

summary(model, input_size=(1, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 61, 61]             544
         MaxPool2d-2           [-1, 32, 30, 30]               0
            Conv2d-3           [-1, 16, 27, 27]           8,208
         MaxPool2d-4           [-1, 16, 13, 13]               0
            Linear-5                   [-1, 10]          27,050
            Linear-6                    [-1, 1]              11
Total params: 35,813
Trainable params: 35,813
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 1.24
Params size (MB): 0.14
Estimated Total Size (MB): 1.39
----------------------------------------------------------------


In [20]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=10, batch_first=True)
        self.lin1 = nn.Linear(10, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, input):
        lstm_out, _ = self.lstm(input)
        last_hidden_state = lstm_out[:, -1, :]
        h1 = F.relu(self.lin1(last_hidden_state))
        output = torch.sigmoid(self.output(h1))
        return output

model = RNN()

#LSTM-el meggyűlt a baja a torchsummary-nak
torchinfo.summary(model, input_size=(1,100, 1))

Layer (type:depth-idx)                   Output Shape              Param #
RNN                                      [1, 1]                    --
├─LSTM: 1-1                              [1, 100, 10]              520
├─Linear: 1-2                            [1, 10]                   110
├─Linear: 1-3                            [1, 1]                    11
Total params: 641
Trainable params: 641
Non-trainable params: 0
Total mult-adds (M): 0.05
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [26]:
text = 'The quick brown fox jumped over the lazy dog.'

tokenizer = get_tokenizer('basic_english')
result = tokenizer(text)
result

['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog', '.']

In [39]:
df = pd.read_csv('https://github.com/yarathealmighty/dumps/blob/main/data/not_na_tags/latin_374_w2v.csv?raw=true')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [46]:
vocab = build_vocab_from_df(df, 'Question')

print('Vocabulary Size:', len(vocab))
print('Top Words:', vocab.most_common(50))

min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

Vocabulary Size: 57518
Top Words: [('Latin', 7672), ('would', 3867), ('like', 2709), ('word', 2621), ('one', 2091), ('translation', 1862), ('Im', 1857), ('English', 1745), ('words', 1647), ('used', 1628), ('know', 1560), ('Greek', 1519), ('use', 1514), ('question', 1495), ('meaning', 1421), ('say', 1414), ('something', 1364), ('et', 1317), ('also', 1216), ('mean', 1178), ('verb', 1176), ('could', 1156), ('seems', 1147), ('two', 1144), ('est', 1141), ('phrase', 1140), ('sentence', 1124), ('example', 1113), ('translate', 1058), ('way', 1050), ('first', 1015), ('correct', 1008), ('find', 950), ('see', 836), ('sense', 826), ('dont', 813), ('form', 807), ('case', 800), ('found', 793), ('think', 791), ('means', 790), ('make', 778), ('different', 763), ('following', 722), ('name', 704), ('want', 689), ('noun', 686), ('good', 678), ('might', 672), ('understand', 663)]
25297
